# Install packages


In [ ]:
import numpy
array = numpy.array([4,2,7,1])
order = array.argsort()
ranks = order.argsort()
print(order)
print(ranks)

[3 1 0 2]
[2 1 3 0]


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.5.0 bitsandbytes==0.40.2 trl==0.4.7 langchain
!pip install -q --no-cache-dir transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
# Make sure a GPU is available
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU")

GPU is available


# Imports

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import pandas as pd
from string import Template
from pathlib import Path

import os

import warnings
warnings.simplefilter("ignore")

from tqdm.notebook import tqdm

# for training
from peft import LoraConfig, get_peft_model, LoraModel, PeftModel
from transformers import TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

# for traing set
from datasets import load_dataset
from langchain.prompts import PromptTemplate
import matplotlib.pyplot as plt
import bitsandbytes as bnb
import numpy as np

from IPython.display import Markdown, display

In [ ]:
# Log in to huggingface to load/push fine-tuned models in hub
from huggingface_hub import notebook_login

notebook_login()

# Load model and tokenizer


In [ ]:
# Change model_name to the model of your choice
# This can be either name of the model on huggingface (requires internet) or path to the model
# model_name = "/kaggle/input/llama2-7b-hf/Llama2-7b-hf"

# Tried the first three, failed at the training step
# model_name = "NousResearch/llama-2-7b-chat-hf"
# model_name = "guardrail/llama-2-7b-guanaco-instruct-sharded"
# model_name = "abhishek/llama-2-7b-hf-small-shards"

# model_name = "lmsys/vicuna-7b-v1.5"
# model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtyp=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
# this should be set as False for finetuning
model.config.use_cache = False

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# For llama2 models
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


# For Vicuna model
#tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)

tokenizer.pad_token = tokenizer.eos_token

# Fine-tuning with QLora


### Prepare training data

In [ ]:
# load training data
train_dataset = load_dataset("csv", data_files="/content/thesis_questions_train.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Prepare template - alternatively add "###Instruction:" etc
template = """Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

Question: {prompt}\n
A) {a}\n
B) {b}\n
C) {c}\n
D) {d}\n
E) {e}\n

### Answer: {answer}"""

prompt = PromptTemplate(template=template, input_variables=['prompt', 'a', 'b', 'c', 'd', 'e', 'answer'])

In [ ]:
# Display sample to see template
sample = train_dataset['train'][1]
display(Markdown(prompt.format(prompt=sample['prompt'],
                               a=sample['A'],
                               b=sample['B'],
                               c=sample['C'],
                               d=sample['D'],
                               e=sample['E'],
                               answer=sample['answer'])))

Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

Question: Which of the following is not an indication for liver transplantation?

A) post-inflammatory liver cirrhosis.

B) Budd-Chiari syndrome.

C) Wilson’s disease.

D) primary and secondary biliary liver cirrhosis.

E) disseminated hepatocellular carcinoma.


### Answer: E

In [ ]:
# Format an example question into the prompt format
def format_text(example):
    """ fill inputs in prompt for a sample  """
    text = prompt.format(prompt=example['prompt'],
                         a=example['A'],
                         b=example['B'],
                         c=example['C'],
                         d=example['D'],
                         e=example['E'],
                         answer=example['answer'])
    return {"text": text}

In [ ]:
# Make the training dataset to have this format
train_dataset = train_dataset.map(format_text)

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

### Set up training arguments

In [ ]:
# Check model structure
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [ ]:
def find_linear_layers(model):
    """ Find linear layers in the given transformer model """
    lora_module_names = set()
    for name, module in model.named_modules():
        # 4 bits for qlora
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)


# Only apply QLoRA on linear layers of the model
target_modules = find_linear_layers(model)

# For llama 2 (they need different target module)
qlora_config = LoraConfig(
    r=16,  # dimension of the updated matrices
    lora_alpha=64,  # parameter for scaling
    target_modules=target_modules, # this chooses on which modules inside our model needs to be amended with LoRA matrices
    lora_dropout=0.1,  # dropout probability for layers
    bias="none",
    task_type="CAUSAL_LM",
)

LoRA module names: ['up_proj', 'k_proj', 'v_proj', 'gate_proj', 'o_proj', 'q_proj', 'down_proj']


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraModel, get_peft_model

# Print the number of trainable parameters (others kept frozen)
lora_model = get_peft_model(model, qlora_config)
# lora_model = LoraModel(qlora_config, model)
print_trainable_parameters(lora_model)

trainable params: 39976960 || all params: 3540389888 || trainable%: 1.1291682911958425


In [ ]:
# Change the saved model name here
# output_dir = "./vicuna-7b-qlora-lek-train-4-epochs"
output_dir = "./llama2-7b-chat-qlora-lek-train-4-epochs-recheck"

# Set up training arguments
# "max_steps=1" is just for testing execution
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2, # Set to 2 to increase the batch size without increasing the memory requirement
    learning_rate=2e-5, # Lower the lr to prevent overshooting
    logging_steps=10,
    logging_strategy="steps",
    warmup_steps=2,
    num_train_epochs=4,
    # max_steps=1, # This overrides "num_train_epochs"
    optim="paged_adamw_8bit",
    fp16=True,
    save_total_limit=1,  # The number of saved checkpoints, can be increased, but beware of kaggle notebook output size limit
    report_to="none",
    push_to_hub=True # Push the model to hub
)

In [ ]:
response_template="Answer:"

supervised_finetuning_trainer = SFTTrainer(
    model,
    train_dataset=train_dataset['train'],
    args=training_args,
    tokenizer=tokenizer,
    peft_config=qlora_config,
    dataset_text_field="text",
    max_seq_length=3000, # The maximum number of tokens that will be considered from each sequence when processing text data
    data_collator=DataCollatorForCompletionOnlyLM(tokenizer=tokenizer,
                                                  response_template=response_template) # Train on completions
)

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

In [ ]:
supervised_finetuning_trainer.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.


Step,Training Loss
10,2.711200
20,1.604400
30,1.642400
40,1.331200
50,1.724700
60,1.532900
70,1.360400
80,1.279900
90,1.241500
100,1.306900


TrainOutput(global_step=388, training_loss=0.8295926360279015, metrics={'train_runtime': 2987.7143, 'train_samples_per_second': 1.044, 'train_steps_per_second': 0.13, 'total_flos': 2.686469409010483e+16, 'train_loss': 0.8295926360279015, 'epoch': 3.98})

### Save model

In [ ]:
supervised_finetuning_trainer.push_to_hub()
# supervised_finetuning_trainer.push_to_hub("yy0514/vicuna-7b-qlora-lek-train")

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yy0514/llama2-7b-chat-qlora-lek-train-4-epochs-recheck/commit/7cf34edfc9a8834a1c906a2a6d1536d417826529', commit_message='End of training', commit_description='', oid='7cf34edfc9a8834a1c906a2a6d1536d417826529', pr_url=None, pr_revision=None, pr_num=None)

# Prepare for Inference

### Prepare test set

In [ ]:
# Same prompt as before
template = """Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

Question: {prompt}\n
A) {a}\n
B) {b}\n
C) {c}\n
D) {d}\n
E) {e}\n

### Answer: {answer}"""

prompt = PromptTemplate(template=template, input_variables=['prompt', 'a', 'b', 'c', 'd', 'e', 'answer'])

In [ ]:
# We don't have answers for test
def format_text_test(example):
    text = prompt.format(prompt=example['prompt'],
                         a=example['A'],
                         b=example['B'],
                         c=example['C'],
                         d=example['D'],
                         e=example['E'],
                         answer='')
    return {"text": text}


test_dataset = load_dataset("csv", data_files="/content/thesis_questions_test.csv")
test_dataset = test_dataset.map(format_text_test)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
# Create a test set dataframe
csv_file_path = '/content/thesis_questions_test.csv'

# Replace these with the actual column names you want to keep
columns_to_keep = ['index', 'answer']

# Read the CSV file using pandas and select only the specified columns
test_df = pd.read_csv(csv_file_path, usecols=columns_to_keep)

# Now df will contain only the columns listed in columns_to_keep
print(test_df.head())

   index answer
0      1      C
1      2      A
2      3      B


### Predict the next token and do perplexity ranking

In [ ]:
# Define the metric of perplexity
from torch import nn

class Perplexity(nn.Module):
    def __init__(self, reduce: bool = True):
        super().__init__()
        self.loss_fn = nn.CrossEntropyLoss() # CE loss is the average negative log-likelihood of the next token appearing given the input sequence
        self.reduce = reduce # Control whether the computed perplexity should be reduced (averaged) across all examples in a batch

    def forward(self, logits, labels):
        shift_logits = logits[..., :-1, :].contiguous() # Remove the last token's logits because there is no next token to predict after the last one
        shift_labels = labels[..., 1:].contiguous() # Shift the labels by one position to the left, aligning each token with its subsequent token as the label for prediction

        perplexity = []
        for i in range(labels.shape[0]):
            perplexity.append(self.loss_fn(shift_logits[i], shift_labels[i]))
        perplexity = torch.stack(perplexity, dim=0)
        if self.reduce:
            perplexity = torch.mean(perplexity)
        return perplexity

perp = Perplexity()

In [ ]:
def evaluate_model_on_dataset(model, tokenizer, dataset, device="cuda"):
    model.eval()
    preds = []
    perplexities_ordered = []  # List to store ordered perplexity values for each option

    for idx in tqdm(range(len(dataset["train"])), total=len(dataset["train"])):
        with torch.no_grad():
            # Prepare the sample inputs
            cols = ["A", "B", "C", "D", "E"]
            perps = []
            samples = [dataset['train'][idx]['text'] + col for col in cols]

            # Tokenize the samples
            inputs = tokenizer(samples, return_tensors="pt", add_special_tokens=False, padding=True, truncation=True).to(device)

            # Get model output
            print(inputs)
            output = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
            logits = output.logits
            labels = inputs["input_ids"] # Set up the ground truth for the next-token prediction
            labels.masked_fill_(~inputs["attention_mask"].bool(), -100) # Mask out the tokens that the model should not consider for loss calculation, only keep the option label

            # Calculate perplexity for each option
            perp = Perplexity().to(device)
            for j in range(len(cols)):
                p = perp(logits[j].unsqueeze(0), labels[j].unsqueeze(0))
                perps.append(p.item())

            # Memory management: delete variables no longer needed
            del inputs, labels, output, p

            # Sort the indices based on perplexity and get the corresponding options and perplexities
            sorted_indices = np.argsort(perps)
            predictions = [cols[i] for i in sorted_indices]
            sorted_perps = [perps[i] for i in sorted_indices]

            preds.append(predictions)
            perplexities_ordered.append(sorted_perps)

    return preds, perplexities_ordered


# Inference

### Inference with pre-trained model

In [ ]:
old_preds, old_perplexities_ordered = evaluate_model_on_dataset(model, tokenizer, test_dataset)

  0%|          | 0/3 [00:00<?, ?it/s]

{'input_ids': tensor([[  673,   278,  1494,  ...,   673, 29901,   319],
        [  673,   278,  1494,  ...,   673, 29901,   350],
        [  673,   278,  1494,  ...,   673, 29901,   315],
        [  673,   278,  1494,  ...,   673, 29901,   360],
        [  673,   278,  1494,  ...,   673, 29901,   382]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')}
{'input_ids': tensor([[  673,   278,  1494,  2999,  7348,  1139,   491,  6820,   278,  1556,
          8210,  2933, 29889,   673,   881,   367,   697,  4249,   518, 29909,
         29892,   350, 29892,   315, 29892,   360, 29892,   382, 29962,    13,
            13, 16492, 29901,   530,  8380,  6761,   513,   293,   362,   304,
         18755,   719, 29876,   324,  4848,   338, 29901, 29871,    13,    13,
         29909, 29897, 29871,   338,  5815,   331,   293, 197

In [ ]:
print(old_preds)
print(old_perplexities_ordered)

[['B', 'E', 'C', 'D', 'A'], ['E', 'D', 'B', 'C', 'A'], ['D', 'E', 'C', 'B', 'A']]
[[2.0277669429779053, 2.0349390506744385, 2.0385677814483643, 2.041001081466675, 2.041065216064453], [2.045328378677368, 2.0605595111846924, 2.0621352195739746, 2.0697507858276367, 2.0736899375915527], [2.271695852279663, 2.2923758029937744, 2.3017969131469727, 2.3033478260040283, 2.3143198490142822]]


In [ ]:
def format_prediction(prediction_list, k=3):
    """Formats the top-k predictions into a single string."""
    return ' '.join(prediction_list[:k])

In [ ]:
# Change column names
top1_col_name = 'vicuna_7B_pt_top1'
top3_col_name = 'vicuna_7B_pt_top3'
preds = old_preds

# Create new columns for k=1 and k=3 formatted predictions
test_df[top1_col_name] = [format_prediction(pred, k=1) for pred in preds]
test_df[top3_col_name] = [format_prediction(pred, k=3) for pred in preds]

print(test_df.head())

   index answer vicuna_7B_pt_top1 vicuna_7B_pt_top3
0      1      C                 B             B E C
1      2      A                 E             E D B
2      3      B                 D             D E C


In [ ]:
# Save for safety
test_df.to_csv('/content/test_results.csv', index = False)

### Load the fine-tuned model

In [ ]:
# Directly load the fine-tuned model from AutoModelForCausalLM

# peft_model_id = "yy0514/llama2-7b-qlora-lek-train-4-epochs"
peft_model_id = "yy0514/llama2-7b-chat-qlora-lek-train-4-epochs-recheck"
# peft_model_id = "yy0514/vicuna-7b-qlora-lek-train-4-epochs"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # # Load in 4 bit to save memory - may hurt model performance
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtyp=torch.bfloat16,
)

new_model = AutoModelForCausalLM.from_pretrained(
    peft_model_id,
    quantization_config=bnb_config,
    trust_remote_code=True
)

adapter_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
# Alternatively, load the adapter from the pre-trained model
# model_id = "lmsys/vicuna-7b-v1.5"
# peft_model_id = "yy0514/llama2-7b-chat-qlora-lek-train"

# new_model = model.load_adapter(peft_model_id, adapter_name = "qlora")

In [ ]:
# Load the tokenizer
# model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-2-7b-chat-hf"
# model_name = "lmsys/vicuna-7b-v1.5"

# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True) # for llama2 models
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False) # for Vicuna model
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# The tokenizer would NOT change after fine-tuning, so can load it from the peft model too

# tokenizer = AutoTokenizer.from_pretrained(peft_model_id, trust_remote_code=True) # for llama2 models
# # tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False) # for Vicuna model
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Check the fine-tuned model structure
# new_model

### Inference with fine-tuned model

In [ ]:
new_preds, new_perplexities_ordered = evaluate_model_on_dataset(new_model, tokenizer, test_dataset)

  0%|          | 0/195 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# print(old_preds)
# print(old_perplexities_ordered)

In [ ]:
print(new_preds)
print(new_perplexities_ordered)

[['B', 'A', 'D', 'C', 'E'], ['E', 'B', 'A', 'D', 'C'], ['D', 'A', 'C', 'B', 'E'], ['A', 'B', 'E', 'D', 'C'], ['D', 'A', 'E', 'C', 'B'], ['A', 'C', 'B', 'D', 'E'], ['D', 'A', 'E', 'C', 'B'], ['D', 'E', 'A', 'B', 'C'], ['C', 'A', 'D', 'E', 'B'], ['A', 'C', 'B', 'D', 'E'], ['B', 'D', 'A', 'C', 'E'], ['D', 'E', 'A', 'B', 'C'], ['C', 'B', 'A', 'E', 'D'], ['D', 'A', 'E', 'C', 'B'], ['D', 'A', 'C', 'E', 'B'], ['C', 'A', 'D', 'B', 'E'], ['B', 'E', 'A', 'D', 'C'], ['C', 'E', 'D', 'B', 'A'], ['B', 'D', 'C', 'E', 'A'], ['E', 'D', 'B', 'A', 'C'], ['D', 'A', 'B', 'C', 'E'], ['D', 'C', 'A', 'E', 'B'], ['E', 'B', 'A', 'D', 'C'], ['B', 'A', 'D', 'C', 'E'], ['B', 'A', 'C', 'D', 'E'], ['E', 'A', 'D', 'B', 'C'], ['D', 'E', 'A', 'C', 'B'], ['C', 'D', 'E', 'A', 'B'], ['E', 'B', 'A', 'D', 'C'], ['C', 'B', 'A', 'D', 'E'], ['E', 'A', 'D', 'C', 'B'], ['A', 'C', 'B', 'D', 'E'], ['E', 'A', 'D', 'B', 'C'], ['C', 'A', 'B', 'D', 'E'], ['B', 'A', 'D', 'C', 'E'], ['B', 'A', 'C', 'D', 'E'], ['A', 'B', 'C', 'E', 'D'], 

In [ ]:
def format_prediction(prediction_list, k=3):
    """Formats the top-k predictions into a single string."""
    return ' '.join(prediction_list[:k])

In [ ]:
# Change column names
# top1_col_name = 'vicuna_7B_qlora_top1'
# top3_col_name = 'vicuna_7B_qlora_top3'
# top1_col_name = 'llama2_7B_qlora_top1'
# top3_col_name = 'llama2_7B_qlora_top3'
top1_col_name = 'llama2_chat_7B_qlora_top1'
top3_col_name = 'llama2_chat_7B_qlora_top3'

preds = new_preds

# Create new columns for k=1 and k=3 formatted predictions
test_df[top1_col_name] = [format_prediction(pred, k=1) for pred in preds]
test_df[top3_col_name] = [format_prediction(pred, k=3) for pred in preds]

print(test_df.head())

   index answer llama2_chat_7B_qlora_top1 llama2_chat_7B_qlora_top3
0    199      B                         B                     B A D
1    789      E                         E                     E B A
2    174      B                         D                     D A C
3    467      A                         A                     A B E
4     66      D                         D                     D A E


In [ ]:
# Save the results
test_df.to_csv('/content/test_results.csv', index = False)

# Evaluation

In [ ]:
test_df = pd.read_csv("/content/test_results.csv")
test_df.head()

,index,answer,llama2_chat_7B_qlora_top1,llama2_chat_7B_qlora_top3
0,199,B,B,B A D
1,789,E,E,E B A
2,174,B,D,D A C
3,467,A,A,A B E
4,66,D,D,D A E


In [ ]:
# def MAP_at_3(predictions, true_items):
#     U = len(predictions)
#     map_at_3 = 0.0
#     for u in range(U):
#         user_preds = predictions[u]
#         user_true = true_items[u]
#         user_results = []
#         hit = False
#         for item in user_preds:
#             if item == user_true and not hit:
#                 user_results.append(1)
#                 hit = True  # If correct label has been scored, ignore next ones
#             else:
#                 user_results.append(0)
#         for k in range(min(len(user_preds), 3)):
#             map_at_3 += precision_at_k(user_results, k+1) * user_results[k]
#     return map_at_3 / U


In [ ]:
# Compute the Mean Average Precision (MAP) for the top 3 choices

def precision_at_k(relevance, k):
    assert k <= len(relevance), "k is larger than the length of the list of predictions"
    return sum(relevance[:k]) / k

def MAP_at_3(df, true_col, prediction_col):
    map_score = 0.0
    for _, row in df.iterrows():
        predictions = row[prediction_col].split()  # Split the predictions into a list
        true_answer = row[true_col]
        user_results = [1 if pred == true_answer else 0 for pred in predictions[:3]]

        for k in range(min(len(predictions), 3)):
            # Only consider the first correct label, ignore the rest for MAP calculation
            if user_results[k] == 1:
                map_score += precision_at_k(user_results, k+1)
                break  # Stop after the first correct prediction
    return map_score / len(df)


In [ ]:
def calculate_accuracy(df, true_col, pred_col):
    # Comparing the true answers with the top-1 predictions based on perplexity
    correct_predictions = (df[true_col] == df[pred_col])
    # Calculating the accuracy
    accuracy = correct_predictions.sum() / len(df)
    return accuracy

In [ ]:
# Compute the accuracy
# old_accuracy = calculate_accuracy(test_df, 'answer', old_top1_col)
# print(f"Old Accuracy: {old_accuracy}")

new_accuracy = calculate_accuracy(test_df, 'answer', top1_col_name)
print(f"New Accuracy: {new_accuracy}")

# Compute the MAP@3 scores
# old_map_at_3_score = MAP_at_3(test_df, 'answer', old_top3_col)
# print(f"Old MAP at 3: {old_map_at_3_score}")

new_map_at_3_score = MAP_at_3(test_df, 'answer', top3_col_name)
print(f"New MAP at 3: {new_map_at_3_score}")

New Accuracy: 0.6717948717948717
New MAP at 3: 0.7649572649572648


In [ ]:
# Change column names
# old_top1_col = 'vicuna_7B_pt_top1'
# old_top3_col = 'vicuna_7B_pt_top3'
# new_top1_col = 'vicuna_7B_qlora_top1'
# new_top3_col = 'vicuna_7B_qlora_top3'
old_top1_col = 'llama2_7B_chat_pt_top1'
old_top3_col = 'llama2_7B_chat_pt_top3'
new_top1_col = 'llama2_7B_chat_qlora_top1'
new_top3_col = 'llama2_7B_chat_qlora_top3'

# Compute the accuracy
old_accuracy = calculate_accuracy(test_df, 'answer', old_top1_col)
print(f"Old Accuracy: {old_accuracy}")

new_accuracy = calculate_accuracy(test_df, 'answer', new_top1_col)
print(f"New Accuracy: {new_accuracy}")

# Compute the MAP@3 scores
old_map_at_3_score = MAP_at_3(test_df, 'answer', old_top3_col)
print(f"Old MAP at 3: {old_map_at_3_score}")

new_map_at_3_score = MAP_at_3(test_df, 'answer', new_top3_col)
print(f"New MAP at 3: {new_map_at_3_score}")

KeyError: 'llama2_7B_chat_pt_top1'